This notebook gives a list of basic SPARQL queries which can be run on the chamuça Hindi lexicon. First we will need to import Graph from rdflib and create a Graph object.

In [11]:
from rdflib import Graph

In [6]:
g=Graph()
g.parse("chamuca_hi_lex.rdf", format="turtle")

<Graph identifier=Nfd679e0d227a48369a67f823714c4535 (<class 'rdflib.graph.Graph'>)>

Basic queries: First up, whether a string has source Dalgado

In [8]:
query = """
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry ?etymology
WHERE {
  ?entry a ontolex:LexicalEntry ;
         lexinfo:etymology ?etymology .
  FILTER regex(?etymology, "Source: Dalgado", "i")
}
"""



A query to extract separate etymologies for an entry: first entry

In [17]:
query1 = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry (STR(?etym1) AS ?individualEtymology)
WHERE {
  ?entry lexinfo:etymology ?etymText .
  FILTER CONTAINS(?etymText, "|")
  BIND(STRDT(?etymText, xsd:string) AS ?etymString)

  # Extract first etymology
  BIND(STRBEFORE(?etymString, "|") AS ?etym1)
}
"""

query2= """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry (STR(?etym1) AS ?individualEtymology)
WHERE {
  ?entry lexinfo:etymology ?etymText .
  FILTER CONTAINS(?etymText, "|")
  BIND(STRDT(?etymText, xsd:string) AS ?etymString)

  # Extract first etymology
  BIND(STRAFTER(?etymString, "|") AS ?etym1)
}
"""

query3 = """
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>

SELECT ?entry ?etymology
WHERE {
  ?entry lexinfo:etymology ?etymology .
  FILTER (
    REGEX(STR(?etymology), "^grc\\b") ||
    REGEX(STR(?etymology), "\\|\\s*grc\\b")
  )
}
"""

In [20]:
# Load RDF file
g = Graph()
g.parse("chamuca_hi_lex.rdf", format="turtle")

LEXINFO = Namespace("http://www.lexinfo.net/ontology/2.0/lexinfo#")

query = """
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
SELECT ?entry ?etym
WHERE {
  ?entry lexinfo:etymology ?etym .
}
"""

results = g.query(query)

for row in results:
    etym_str = str(row.etym)
    if re.match(r'^grc\b', etym_str) or re.search(r'\|\s*grc\b', etym_str):
        print(f"{row.entry} → {etym_str}")

http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अनीसून_entry → pt ? anis (Source: Dalgado) | grc ἄνισον ‘anise (Pimpinella anisum)’ (Source: Dalgado) | grc ἄνισον (Source: Wiktionary) >  la anisum ‘anise (Pimpinella anisum)’  (Source: Dalgado) .


In [16]:
for s, o in g.query(query2):
    print(f"{s} --> {o}")

http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अंबर_entry -->  fa عنبر ‘ambergris’ (source:  Wiktionary) | pal ‘ambergris’ mbl > ar عَنْبَر (Source: Dalgado) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अक्तूबर_entry -->  en October (Source: McGregor) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अनीसून_entry -->  grc ἄνισον ‘anise (Pimpinella anisum)’ (Source: Dalgado) | grc ἄνισον (Source: Wiktionary) >  la anisum ‘anise (Pimpinella anisum)’  (Source: Dalgado) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अस्तबल_entry -->  ar اسطبل (Source: McGregor) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अस्पताल_entry -->  en hospital (Source: McGregor) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#आमीन_entry -->  ar آمين ‘amen’ (Source: Dalgado) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#कंदील_entry -->  lat candēla ‘a light made of wax or tallow' > grc κανδήλη ‘oil lamp' > ar قنديل  ‘lamp' (Source: Dalgado) .
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#कप्तान_entry 

In [32]:
query5 = """
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry ?etymology
WHERE {
  ?entry a ontolex:LexicalEntry ;
         lexinfo:etymology ?etymology .
}
"""

query6 = """SELECT ?lemma ?etymology
WHERE {
  ?entry a ontolex:LexicalEntry ;
         lexinfo:etymology ?etymology ;
         ontolex:canonicalForm ?form .
  
  ?form ontolex:writtenRep ?lemma .
}
"""

In [34]:
qres = g.query(query6)

# Python regex filtering (mimics SPARQL FILTER)
pattern = re.compile(r"^(grc\b)|(\>\s*grc\b)|(\|\s*grc\b)")

# Process and print only matching entries
for entry, etymology in qres:
    etymology = str(etymology)
    if pattern.search(etymology):
        print(f"Entry: {entry}")
        print(f"Etymology: {etymology}")
        print("---")

Entry: अनीसून
Etymology: pt ? anis (Source: Dalgado) | grc ἄνισον ‘anise (Pimpinella anisum)’ (Source: Dalgado) | grc ἄνισον (Source: Wiktionary) >  la anisum ‘anise (Pimpinella anisum)’  (Source: Dalgado) .
---
Entry: anīsun
Etymology: pt ? anis (Source: Dalgado) | grc ἄνισον ‘anise (Pimpinella anisum)’ (Source: Dalgado) | grc ἄνισον (Source: Wiktionary) >  la anisum ‘anise (Pimpinella anisum)’  (Source: Dalgado) .
---
Entry: कंदील
Etymology: pt ? candil (Source: Dalgado) | lat candēla ‘a light made of wax or tallow' > grc κανδήλη ‘oil lamp' > ar قنديل  ‘lamp' (Source: Dalgado) .
---
Entry: kandīl
Etymology: pt ? candil (Source: Dalgado) | lat candēla ‘a light made of wax or tallow' > grc κανδήλη ‘oil lamp' > ar قنديل  ‘lamp' (Source: Dalgado) .
---
Entry: साबुन
Etymology: pt sabão (Source: Dalgado) | ine-pro *seyb- > gem-pro *saipǭ [borrowing] > la sāpō [borrowing] > grc σάπων [borrowing] > syc ܨܲܦ݁ܘܿܢܵܐ (Source:Wiktionary)  [borrowing] > ar صَابُون (Source: McGregor) |  ar صَابُون >

A query to extract all multiple etymologies

In [1]:
query4 = """ 
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>

SELECT ?entry ?etymology
WHERE {
  ?entry lexinfo:etymology ?etymology .
  FILTER(CONTAINS(?etymology, "|"))
}

"""

In [9]:
for s, o in g.query(query4):
    new_ety = o.strip().rstrip(".")
    separate_etymologies = [part.strip() for part in new_ety.split('|')]
    for i, ety in enumerate(separate_etymologies, 1):
        print(f"Etymology {i}: {ety}")

Etymology 1: pt ambar (Source: Dalgado)
Etymology 2: fa عنبر ‘ambergris’ (source:  Wiktionary)
Etymology 3: pal ‘ambergris’ mbl > ar عَنْبَر (Source: Dalgado)
Etymology 1: ine-pro *h₃(e)ḱtéh₃ ‘eight’, *h₃ḱt(e)h₃-uó- ‘eighth’ > itc-pro *oktō ‘eight’, *oktāwo- ‘eighth’ (Source: de Vaan) > la octō + -ber ‘pertaining the eigth (month)’ [analogy with la september ‘september’] (Source: Ernout-Meillet) > pt outubro (Source: Wiktionary)
Etymology 2: en October (Source: McGregor)
Etymology 1: pt ? anis (Source: Dalgado)
Etymology 2: grc ἄνισον ‘anise (Pimpinella anisum)’ (Source: Dalgado)
Etymology 3: grc ἄνισον (Source: Wiktionary) >  la anisum ‘anise (Pimpinella anisum)’  (Source: Dalgado)
Etymology 1: pt espital (Source: Wiktionary)
Etymology 2: en hospital (Source: McGregor)
Etymology 1: pt amen (Source: Dalgado)
Etymology 2: ar آمين ‘amen’ (Source: Dalgado)
Etymology 1: pt ? candil (Source: Dalgado)
Etymology 2: lat candēla ‘a light made of wax or tallow' > grc κανδήλη ‘oil lamp' > ar 

http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अंबर_entry --> pt ambar (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अक्तूबर_entry --> pt outubro (Source: Wiktionary) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अनीसून_entry --> pt ? anis (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अस्तबल_entry --> pt estábulo (Source: McGregor) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अस्पताल_entry --> pt espital (Source: Wiktionary) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#आमीन_entry --> pt amen (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#कंदील_entry --> pt ? candil (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#कप्तान_entry --> pt capitão (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#कर्नल_entry --> pt coronel (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#कलापट्टी_entry --> pt calafate (Source: Dalgado) 
http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#क़मीज़_entr

In [16]:
query = """
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry ?matchedStep
WHERE {
?entry a ontolex:LexicalEntry ;
        lexinfo:etymology ?etymology .

# Extract just the step with "Source: Dalgado", stopping at next " |" or at period
BIND(REPLACE(?etymology, ".*?(\\b[^|]*?Source: Dalgado[^|]*?)(\\s*\\|.*|\\s*\\.)$", "$1") AS ?matchedStep)

FILTER CONTAINS(?matchedStep, "Source: Dalgado")
}
"""

In [26]:
query = """
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry ?etymology
WHERE {
  ?entry a ontolex:LexicalEntry ;
         lexinfo:etymology ?etymology .
  FILTER regex(?etymology, "[^|]*Source: Dalgado", "i")
}
"""

In [28]:
query ="""
PREFIX lexinfo: <http://www.lexinfo.net/ontology/2.0/lexinfo#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>

SELECT ?entry ?matchedStep
WHERE {
  ?entry a ontolex:LexicalEntry ;
         lexinfo:etymology ?etymology .

  # Extract just the step with "Source: Dalgado", stopping at next " |" or at period
  BIND(REPLACE(?etymology, ".*?(\\b[^|]*?Source: Dalgado[^|]*?)(\\s*\\|.*|\\s*\\.)$", "$1") AS ?matchedStep)
  
  FILTER CONTAINS(?matchedStep, "Source: Dalgado")
}
"""

In [4]:
print(g.query(query))

for p in g.query(query):
    print(p)

(rdflib.term.URIRef('http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अँग्रेज़_entry'), rdflib.term.Literal('pt inglês (Source: Dalgado) .'))
(rdflib.term.URIRef('http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अंग्रेज़_entry'), rdflib.term.Literal('pt inglês (Source: Dalgado) .'))
(rdflib.term.URIRef('http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अंबर_entry'), rdflib.term.Literal('pt ambar (Source: Dalgado) | fa عنبر ‘ambergris’ (source:  Wiktionary) | pal ‘ambergris’ mbl > ar عَنْبَر (Source: Dalgado) .'))
(rdflib.term.URIRef('http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अचार_entry'), rdflib.term.Literal('pt achar (Source: Dalgado) .'))
(rdflib.term.URIRef('http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अनन्नास_entry'), rdflib.term.Literal('tpn naná ‘pineapple’ (Source: Wiktionary) > pt ananás ‘pineapple’ (Source: Dalgado) .'))
(rdflib.term.URIRef('http://lari-datasets.ilc.cnr.it/chamuca_hi_lex#अनीसून_entry'), rdflib.term.Literal('pt ? anis (Source: Dalgado) | grc ἄνισον ‘anise (Pimpi